<font color=lightblue size=20>Projeto Telecom X: Análise de evasão de clientes

# 📌 Extração

In [71]:
import pandas as pd
import numpy as np
import requests
import json

"""Optei por usar a url do arquivo raw do GitHub a fim de facilitar a utilização
do notebook por um usuário com poucos conhecimentos de importação de dados,
visto que dessa forma seria necessário apenas executar o código ao invés de
precisar importar o arquivo com os dados."""

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

response = requests.get(url) # Requests usado para carregar os dados json
response.raise_for_status() # Lança uma exceção para códigos de status HTTP ruins (4xx ou 5xx)

# Carrega o conteúdo JSON como um objeto Python
dados_json = json.loads(response.text)

# Normaliza os dados
df = pd.json_normalize(dados_json, sep='_')

df.tail()

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65
7266,9995-HOTOH,No,Male,0,Yes,Yes,63,No,No phone service,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Electronic check,59.00,3707.6


#🔧 Transformação

In [95]:
# Substituindo strings vazias por NAN
df['account_Charges_Total'] = df['account_Charges_Total'].replace(' ', np.nan)
# Alterando o tipo de dados da coluna de str para float
df['account_Charges_Total'] = df['account_Charges_Total'].astype(np.float64)


for iten in df.columns: # verificando as colunas
   print(f'{iten:>30}: {df[iten].dtype} => {pd.unique(df[iten])}')

                    customerID: object => ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
                         Churn: object => ['No' 'Yes' '']
               customer_gender: object => ['Female' 'Male']
        customer_SeniorCitizen: int64 => [0 1]
              customer_Partner: object => ['Yes' 'No']
           customer_Dependents: object => ['Yes' 'No']
               customer_tenure: int64 => [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
            phone_PhoneService: object => ['Yes' 'No']
           phone_MultipleLines: object => ['No' 'Yes' 'No phone service']
      internet_InternetService: object => ['DSL' 'Fiber optic' 'No']
       internet_OnlineSecurity: object => ['No' 'Yes' 'No internet service']
         internet_OnlineBackup: object => ['Yes' 'No' 'No 

In [81]:
df.duplicated().sum() # Conferindo duplicados


Número total de linhas duplicadas (excluindo a primeira ocorrência): 0
Número total de linhas que possuem alguma duplicata: 0


#📊 Carga e análise

#📄Relatorio Final

#### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa
* `gender`: gênero (masculino e feminino)
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico
* `MultipleLines`: assisnatura de mais de uma linha de telefone
* `InternetService`: assinatura de um provedor internet
* `OnlineSecurity`: assinatura adicional de segurança online
* `OnlineBackup`: assinatura adicional de backup online
* `DeviceProtection`: assinatura adicional de proteção no dispositivo
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo
* `StreamingMovies`: assinatura de streaming de filmes
* `Contract`: tipo de contrato
* `PaperlessBilling`: se o cliente prefere receber online a fatura
* `PaymentMethod`: forma de pagamento
* `Charges.Monthly`: total de todos os serviços do cliente por mês
* `Charges.Total`: total gasto pelo cliente